In [1]:
import pandas as pd
import json
import os

### Recompute pass@1 scores

In [2]:
model = "Qwen--Qwen2.5-7B-Instruct-1M" # "allenai--OLMo-2-1124-7B-Instruct" #  # #   # #

In [3]:
def is_correct(x, subset="base"):
    if subset == "base":
        return True if x['base_status'] == "pass" else False
    elif subset == "plus":
        return True if x['base_status'] == "pass" and x['plus_status'] == "pass" else False
    else:
        raise NotImplementedError

In [4]:
baseline_file = f"results/q_a_0/{model}_errors.csv"
baseline_df = pd.read_csv(baseline_file)
baseline_df = baseline_df.sort_values(by='task_id')
print(len(baseline_df))
baseline_df['baseline_base_correct'] = baseline_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
baseline_df['baseline_plus_correct'] = baseline_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)
# print(baseline_df.baseline_base_correct.value_counts())
# print(baseline_df.baseline_plus_correct.value_counts())
baseline_df.head()

114


,task_id,solution,base_status,plus_status,base_fail_tests,plus_fail_tests,gt_solution,question_prompt,entry_point,baseline_base_correct,baseline_plus_correct
20,Mbpp/103,"def eulerian_num(n, m):\n # Base case\n ...",fail,fail,"[[3, 1]]","[[1, 1]]","\ndef eulerian_num(n, m): \n\tif (m >= n or n ...","""""""\nWrite a function to find the Eulerian num...",eulerian_num,False,False
21,Mbpp/106,"def add_lists(list1, tuple1):\n # Convert t...",fail,fail,"[[[5, 6, 7], [9, 10]]]","[[[1, [2, 3], {'a': 4}], [5, [6, 7], {'b': 8}]]]","\ndef add_lists(test_list, test_tup):\n retur...","""""""\nWrite a function to append the given list...",add_lists,False,False
16,Mbpp/109,"def odd_Equivalents(binary_str, rotations):\n ...",fail,fail,[],[],"\ndef odd_Equivalent(s,n): \n count=0\n ...","""""""\nWrite a python function to find the numbe...",odd_Equivalent,False,False
10,Mbpp/113,def check_integer(string):\n # Check if the...,pass,fail,[],[['']],\ndef check_integer(text):\n text = text.strip...,"""""""\nWrite a function to check if a string rep...",check_integer,True,False
31,Mbpp/123,def amicable_numbers_sum(n):\n # Helper fun...,fail,fail,[[999]],[[10000]],\ndef div_sum(num):\n res = 1\n i = 2\n ...,"""""""\nWrite a function to sum all amicable numb...",amicable_numbers_sum,False,False


In [5]:
# calculate baseline accuracy
baseline_base_correct = baseline_df.baseline_base_correct.value_counts()[True]
baseline_plus_correct = baseline_df.baseline_plus_correct.value_counts()[True] if True in baseline_df.baseline_plus_correct.value_counts() else 0
baseline_base_accuracy = baseline_base_correct / len(baseline_df)
baseline_plus_accuracy = baseline_plus_correct / len(baseline_df)
print(f"Baseline accuracy: {baseline_base_accuracy}, {baseline_plus_accuracy}")

Baseline accuracy: 0.30701754385964913, 0.0


In [6]:
# get all folders in results/a1
folders = [f for f in os.listdir("results/a1") if os.path.isdir(os.path.join("results/a1", f))]
print(folders)
print(len(folders))
output_file = f"results/metrics/{model}_scores.csv"

['mbpp_just_retrieval_as_feedback', 'mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_with_feedback_gpt-3.5-turbo', 'mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question', 'mbpp_just_feedback_gpt-3.5-turbo']
8


In [7]:
all_rows = []
for setup in folders:

    results_file = f"results/a1/{setup}/{model}_vllm_temp_0.0.eval_results.json"
    model_results = json.load(open(results_file))
    results = [result[0] for result in model_results['eval'].values()]
    exp_df = pd.DataFrame(results)

    def get_baseline_correct(x, subset="base"):
        if x.task_id not in set(baseline_df.task_id):
            return True
        if subset == "base":
            return baseline_df[baseline_df.task_id == x.task_id].baseline_base_correct.values[0]
        elif subset == "plus":
            return baseline_df[baseline_df.task_id == x.task_id].baseline_plus_correct.values[0]
        else:
            raise NotImplementedError

    exp_df['baseline_base_correct'] = exp_df.apply(lambda x: get_baseline_correct(x, subset='base'), axis=1)
    exp_df['baseline_plus_correct'] = exp_df.apply(lambda x: get_baseline_correct(x, subset='plus'), axis=1)

    exp_df['base_correct'] = exp_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
    exp_df['plus_correct'] = exp_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)
    # print(exp_df.head())

    # get the additional correct examples in exp_df that are wrong in baseline
    exp_add_base_correct = len(exp_df[exp_df.base_correct & ~exp_df.baseline_base_correct])
    exp_add_plus_correct = len(exp_df[exp_df.plus_correct & ~exp_df.baseline_plus_correct])
    print("Wrong to correct:", exp_add_base_correct, exp_add_plus_correct)

    # check correct --> wrong
    # base_correct_to_wrong = len(exp_df[~exp_df.base_correct &~exp_df.baseline_base_correct])
    # plus_correct_to_wrong = len(exp_df[~exp_df.plus_correct &~exp_df.baseline_plus_correct])
    # print("Correct to wrong:", base_correct_to_wrong, plus_correct_to_wrong)

    exp_base_accuracy = (baseline_base_correct + exp_add_base_correct) / len(exp_df)
    exp_plus_accuracy = (baseline_plus_correct + exp_add_plus_correct) / len(exp_df)
    print(f"{setup} acc: {exp_base_accuracy}, {exp_plus_accuracy}")

    row = {'setup': setup, 'model': model, 'base_accuracy': exp_base_accuracy, 'plus_accuracy': exp_plus_accuracy}
    # print(row)
    all_rows.append(row)
baseline_row = {'setup': 'baseline', 'model': model, 'base_accuracy': baseline_base_accuracy, 'plus_accuracy': baseline_plus_accuracy}
all_rows.append(baseline_row)


Wrong to correct: 6 8
mbpp_just_retrieval_as_feedback acc: 0.10846560846560846, 0.021164021164021163
Wrong to correct: 5 10
mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.10582010582010581, 0.026455026455026454
Wrong to correct: 10 10
mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.11904761904761904, 0.026455026455026454
Wrong to correct: 18 21
mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.1402116402116402, 0.05555555555555555
Wrong to correct: 12 10
mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question acc: 0.12433862433862433, 0.026455026455026454
Wrong to correct: 25 17
mbpp_with_feedback_gpt-3.5-turbo acc: 0.15873015873015872, 0.04497354497354497
Wrong to correct: 5 4
mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question acc: 0.10582010582010581, 0.010582010582010581
Wrong to correct: 28 19
mbpp_just_feedback_gpt-3.5-turbo acc: 0.16666666666666666, 0.05026455026455026


In [8]:
# save all rows to a csv file
df = pd.DataFrame(all_rows)
df.to_csv(output_file, index=False)
df.head()

,setup,model,base_accuracy,plus_accuracy
0,mbpp_just_retrieval_as_feedback,Qwen--Qwen2.5-7B-Instruct-1M,0.108466,0.021164
1,mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_q...,Qwen--Qwen2.5-7B-Instruct-1M,0.105820,0.026455
2,mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_q...,Qwen--Qwen2.5-7B-Instruct-1M,0.119048,0.026455
3,mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_ch...,Qwen--Qwen2.5-7B-Instruct-1M,0.140212,0.055556
4,mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_n...,Qwen--Qwen2.5-7B-Instruct-1M,0.124339,0.026455


### Compute helpfulness, leakage frequency and non-leaking helpfulness

In [9]:
output_file = f"results/pass_at_k_only_retried_instances/{model}_eval_metrics.csv"
print(output_file)

results/pass_at_k_only_retried_instances/Qwen--Qwen2.5-7B-Instruct-1M_eval_metrics.csv


In [10]:
exp_ids = [
    "Teacher Feedback",
    "Teacher Feedback (No Cheat No Question)",
    "Teacher Feedback with RAG",
    "RAG Feedback",
]

leak_exps = [
    'mbpp_just_feedback_gpt-3.5-turbo',
    'mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question', 
    'mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question',
    'mbpp_just_retrieval_as_feedback', 
    
]

actual_exps = [
    'mbpp_with_feedback_gpt-3.5-turbo', 
    'mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question', 
    'mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question',
    'mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question', 
]


In [11]:
all_rows = []
for exp_id, leak_exp, actual_exp in zip(exp_ids, leak_exps, actual_exps):
    print(f"{exp_id} (actual = {actual_exp}, leak exp = {leak_exp}):")
    
    leak_exp_file = f"results/a1/{leak_exp}/{model}_vllm_temp_0.0.eval_results.json"
    actual_exp_file = f"results/a1/{actual_exp}/{model}_vllm_temp_0.0.eval_results.json"

    leak_exp_results = json.load(open(leak_exp_file))
    actual_exp_results = json.load(open(actual_exp_file))

    leak_exp_results = [result[0] for result in leak_exp_results['eval'].values()]
    actual_exp_results = [result[0] for result in actual_exp_results['eval'].values()]

    leak_exp_df = pd.DataFrame(leak_exp_results)
    actual_exp_df = pd.DataFrame(actual_exp_results)

    leak_exp_df['base_correct'] = leak_exp_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
    leak_exp_df['plus_correct'] = leak_exp_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)

    actual_exp_df['baseline_base_correct'] = actual_exp_df.apply(lambda x: get_baseline_correct(x, subset='base'), axis=1)
    actual_exp_df['baseline_plus_correct'] = actual_exp_df.apply(lambda x: get_baseline_correct(x, subset='plus'), axis=1)

    actual_exp_df['base_correct'] = actual_exp_df.apply(lambda x: is_correct(x, subset="base"), axis=1)
    actual_exp_df['plus_correct'] = actual_exp_df.apply(lambda x: is_correct(x, subset="plus"), axis=1)

    actual_exp_df['base_leaked'] = actual_exp_df.apply(lambda x: leak_exp_df[leak_exp_df.task_id == x.task_id].base_correct.values[0] & ~x.baseline_base_correct, axis=1)
    actual_exp_df['plus_leaked'] = actual_exp_df.apply(lambda x: leak_exp_df[leak_exp_df.task_id == x.task_id].plus_correct.values[0] & ~x.baseline_plus_correct, axis=1)

    # helpfulness 
    actual_exp_df['base_helpful'] = actual_exp_df.apply(lambda x: x.base_correct & ~x.baseline_base_correct, axis=1)
    actual_exp_df['plus_helpful'] = actual_exp_df.apply(lambda x: x.plus_correct & ~x.baseline_plus_correct, axis=1)
    # print(actual_exp_df.head())

    # non leaking helpfulness 
    actual_exp_df['base_nonleak_helpful'] = actual_exp_df.apply(lambda x: x.base_correct & ~x.baseline_base_correct & ~x.base_leaked, axis=1)
    actual_exp_df['plus_nonleak_helpful'] = actual_exp_df.apply(lambda x: x.plus_correct & ~x.baseline_plus_correct & ~x.plus_leaked, axis=1)

    row = {"exp_id": exp_id, "model": model}

    for subset in ['base', 'plus']:
        total_cnt = len(actual_exp_df[actual_exp_df[f'baseline_{subset}_correct'] == 0])
        helpful = round(len(actual_exp_df[actual_exp_df[f'{subset}_helpful'] == 1]) / total_cnt, 3)
        nonleak_helpful = round(len(actual_exp_df[actual_exp_df[f'{subset}_nonleak_helpful'] == 1]) / total_cnt, 3)
        leak_freq = round(len(actual_exp_df[actual_exp_df[f'{subset}_leaked'] == 1]) / total_cnt, 3)
        print(f"{subset} helpful: {helpful}, leakage: {leak_freq}, nonleak helpful: {nonleak_helpful}")
        row[f"{subset}_helpfulness"] = helpful
        row[f"{subset}_nonleak_helpfulness"] = nonleak_helpful
        row[f"{subset}_leakage_freq"] = leak_freq
    
   
    all_rows.append(row)

df = pd.DataFrame(all_rows)
df.to_csv(output_file, index=False)
df.head()

Teacher Feedback (actual = mbpp_with_feedback_gpt-3.5-turbo, leak exp = mbpp_just_feedback_gpt-3.5-turbo):
base helpful: 0.316, leakage: 0.354, nonleak helpful: 0.013
plus helpful: 0.149, leakage: 0.167, nonleak helpful: 0.009
Teacher Feedback (No Cheat No Question) (actual = mbpp_with_feedback_gpt-3.5-turbo_no_cheat_no_question, leak exp = mbpp_just_feedback_gpt-3.5-turbo_no_cheat_no_question):
base helpful: 0.127, leakage: 0.063, nonleak helpful: 0.127
plus helpful: 0.088, leakage: 0.088, nonleak helpful: 0.079
Teacher Feedback with RAG (actual = mbpp_with_rag_teacher_feedback_gpt-3.5-turbo_no_cheat_no_question, leak exp = mbpp_just_feedback_rag_teacher_gpt-3.5-turbo_no_cheat_no_question):
base helpful: 0.152, leakage: 0.063, nonleak helpful: 0.127
plus helpful: 0.088, leakage: 0.035, nonleak helpful: 0.079
RAG Feedback (actual = mbpp_retrieval_as_feedback_gpt-3.5-turbo_no_cheat_no_question, leak exp = mbpp_just_retrieval_as_feedback):
base helpful: 0.228, leakage: 0.076, nonleak hel

,exp_id,model,base_helpfulness,base_nonleak_helpfulness,base_leakage_freq,plus_helpfulness,plus_nonleak_helpfulness,plus_leakage_freq
0,Teacher Feedback,Qwen--Qwen2.5-7B-Instruct-1M,0.316,0.013,0.354,0.149,0.009,0.167
1,Teacher Feedback (No Cheat No Question),Qwen--Qwen2.5-7B-Instruct-1M,0.127,0.127,0.063,0.088,0.079,0.088
2,Teacher Feedback with RAG,Qwen--Qwen2.5-7B-Instruct-1M,0.152,0.127,0.063,0.088,0.079,0.035
3,RAG Feedback,Qwen--Qwen2.5-7B-Instruct-1M,0.228,0.190,0.076,0.184,0.149,0.070


In [12]:
output_file

'results/pass_at_k_only_retried_instances/Qwen--Qwen2.5-7B-Instruct-1M_eval_metrics.csv'